In [ ]:
#import os
#import cv2
#import pickle
#import numpy as np
#import pandas as pd
#from skimage.feature import graycomatrix, graycoprops
#from PIL import Image
#import matplotlib.pylab as plt
#import mahotas as mh
#from sklearn import svm
#from sklearn.svm import SVC
#from sklearn.model_selection import train_test_split
#from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
#from sklearn.utils.class_weight import compute_class_weight
#from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
#from scipy.stats import uniform, loguniform
#import seaborn as sns
#import tensorflow as tf
#from sklearn.decomposition import PCA
#from sklearn.preprocessing import StandardScaler
#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import SMOTE
#from sklearn.pipeline import Pipeline
#from keras._tf_keras.keras.preprocessing.image import ImageDataGenerator
#from keras._tf_keras.keras.applications import EfficientNetB0
#from keras._tf_keras.keras.layers import Dense, GlobalAveragePooling2D, Dropout
#from keras._tf_keras.keras.models import Model
#from keras._tf_keras.keras.optimizers import Adam
#from keras._tf_keras.keras.callbacks import EarlyStopping
#from sklearn.model_selection import train_test_split
#import shutil
#import torch
#import torch.nn as nn
#import torch.optim as optim
#from torchvision import datasets, models, transforms
#from torch.utils.data import DataLoader, WeightedRandomSampler

In [ ]:
import numpy as np
import pandas as pd
from skimage.feature import graycomatrix, graycoprops
from PIL import Image

In [ ]:
def quantize_image(image, levels = 16):
    grayscale_image = image.convert("L")
    quantized_image = np.array(grayscale_image) // (256 // levels)
    return quantized_image

In [ ]:
def compute_cooccurrence_matrices(q_img, distances=[1, 2, 4, 8, 16, 32], angles=[0]):
    cooccurrence_matrices = graycomatrix(q_img, distances, angles, levels=16, symmetric=True, normed=True)
    return cooccurrence_matrices


In [ ]:
def compute_glcm(q_img, dists=[1, 2 ,4, 8, 16, 32], angles=[0]):
    glcm = graycomatrix(q_img, distances=dists, angles=angles, levels=16, symmetric=True, normed=True)
    return glcm

In [ ]:
def compute_haralick_features(glcm):
    contrast = graycoprops(glcm, 'contrast').flatten()
    homogeneity = graycoprops(glcm, 'homogeneity').flatten()
    
    # Entropy calculation
    glcm_sum = glcm.sum(axis=(0, 1))
    norm_glcm = glcm / glcm_sum
    entropy = -np.sum(norm_glcm * np.log(norm_glcm + 1e-10), axis=(0, 1)).flatten()
    
    features = {
        'contrast': contrast,
        'homogeneity': homogeneity,
        'entropy': entropy
    }
    return features

In [ ]:
img_data = []

for index, row in table_data.iterrows():
    cell_id = row['cell_id']
    img_path = os.path.join(CLASSES_PATH, row['bethesda_system'], f"{row['cell_id']}.png")
    img_class = row['bethesda_system']

    image = Image.open(img_path)

    if image is not None:
        q_img = quantize_image(image)
        glcm = compute_glcm(q_img)
        haralick_features = compute_haralick_features(glcm)

        img_data.append ({
            'cell_id': cell_id,
            'image_path': img_path,
            'contrast_1': haralick_features['contrast'][0],
            'contrast_2': haralick_features['contrast'][1],
            'contrast_4': haralick_features['contrast'][2],
            'contrast_8': haralick_features['contrast'][3],
            'contrast_16': haralick_features['contrast'][4],
            'contrast_32': haralick_features['contrast'][5],
            'homogeneity_1': haralick_features['homogeneity'][0],
            'homogeneity_2': haralick_features['homogeneity'][1],
            'homogeneity_4': haralick_features['homogeneity'][2],
            'homogeneity_8': haralick_features['homogeneity'][3],
            'homogeneity_16': haralick_features['homogeneity'][4],
            'homogeneity_32': haralick_features['homogeneity'][5],
            'entropy_1': haralick_features['entropy'][0],
            'entropy_2': haralick_features['entropy'][1],
            'entropy_4': haralick_features['entropy'][2],
            'entropy_8': haralick_features['entropy'][3],
            'entropy_16': haralick_features['entropy'][4],
            'entropy_32': haralick_features['entropy'][5],
            'img_class': img_class
        })
        
dataset = pd.DataFrame(img_data)